In [27]:
import os
import sys
sys.path.append("../../src/")
sys.path.append("../../utilities/")
import dataset_class
from dataset_class import dataset
import data_utils
from importlib import reload
dataset_class = reload(dataset_class)
from dataset_class import dataset
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.special import logsumexp
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from GMM_GBR import GMM_MP
import mrf_util
from mrf import MarkovRandomField
from hmmlearn import hmm

In [6]:
GM12878_signals_names = ['DNase', 'H3K4me2', 'H3K27ac', 'H2A.Z', 'H3K9me3', 'H3K4me1', 'H3K27me3', 'H3K9ac', 'H3K4me3', 'H3K36me3', 'H3K79me2', 'H4K20me1']
GM12878_dataset = dataset('GM12878', 'hg19', 100000, '../../data/GM12878/hg19/Hi-C/GSE63525_GM12878_insitu_primary_30.hic',
                 '../../utilities/juicer_tools_1.22.01.jar', '../../data/GM12878/hg19/Hi-C/res100000',
                 GM12878_signals_names, '../../data/GM12878/hg19/genomic-assays/bin_100000/', 
                  '../../data/GM12878/hg19/res100000_datasets/',
                  '../../data/', '../../utilities/config.json')
GM12878_dataset.load_pos2ind_and_ind2pos_maps('../../data/valid_bins/hg19/GM12878_100000_bins.txt')

In [5]:
'''
signal_path = '../../data/GM12878/hg19/res100000_datasets/signals.txt'
bin_path = '../../data/GM12878/hg19/res100000_datasets/bin.txt'
genomedata_path = '../../data/GM12878/hg19/res100000_datasets/signals.genomedata'
data_utils.InputsAndBins_to_genomedata(signal_path, GM12878_signals_names, bin_path, 'hg19', 100000, genomedata_path)
signal_path = '../../data/GM12878/hg19/res100000_datasets/structural_signals.txt'
bin_path = '../../data/GM12878/hg19/res100000_datasets/bin.txt'
genomedata_path = '../../data/GM12878/hg19/res100000_datasets/structural_signals.genomedata'
signals_names = ['emb{}'.format(e) for e in np.arange(1,9)]
data_utils.InputsAndBins_to_genomedata(signal_path, signals_names, bin_path, 'hg19', 100000, genomedata_path)
'''

In [19]:
#signals_path = "../../alternatives/SPIN-master/data/test_input_chr1.txt"
#bins_path = "../../alternatives/SPIN-master/data/test_bin_chr1.bed"
#hic_path = "../../alternatives/SPIN-master/data/test_Hi-C_chr1.txt"
functional_signals_path = "../../data/GM12878/hg19/res100000_datasets/signals.txt"
structural_signals_path = "../../data/GM12878/hg19/res100000_datasets/structural_signals.txt"
bins_path = "../../data/GM12878/hg19/res100000_datasets/bin.txt"
short_hic_path = "../../data/GM12878/hg19/res100000_datasets/short_range_interactions.txt"
long_hic_path = "../../data/GM12878/hg19/res100000_datasets/long_range_interactions.txt"
k=8
annotation_dir = "../../data/GM12878/hg19/res100000_datasets/annotations/K=8/"

In [13]:
functional_signals = np.loadtxt(functional_signals_path)
structural_signals = np.loadtxt(structural_signals_path)
lengths = GM12878_dataset.get_chunks_lengths()
signals = np.concatenate((functional_signals, structural_signals), axis = 1)

# KMeans

In [21]:
kmeans = KMeans(n_clusters=k, random_state=0).fit(functional_signals)
l = kmeans.labels_
GM12878_dataset.write_annotation(l, 'kmeans_functional', annotation_dir)

kmeans = KMeans(n_clusters=k, random_state=0).fit(structural_signals)
l = kmeans.labels_
GM12878_dataset.write_annotation(l, 'kmeans_structural', annotation_dir)

kmeans = KMeans(n_clusters=k, random_state=0).fit(signals)
l = kmeans.labels_
GM12878_dataset.write_annotation(l, 'kmeans_combined', annotation_dir)

# GMM

In [22]:
gmm = GaussianMixture(n_components=k, random_state=0).fit(functional_signals)
l = gmm.predict(functional_signals)
GM12878_dataset.write_annotation(l, 'GMM_functional', annotation_dir)

gmm = GaussianMixture(n_components=k, random_state=0).fit(structural_signals)
l = gmm.predict(structural_signals)
GM12878_dataset.write_annotation(l, 'GMM_structural', annotation_dir)

gmm = GaussianMixture(n_components=k, random_state=0).fit(signals)
l = gmm.predict(signals)
GM12878_dataset.write_annotation(l, 'GMM_combined', annotation_dir)

# HMM (Viterbi)

In [23]:
gmm_hmm_viterbi = hmm.GaussianHMM(n_components=k, covariance_type = 'full', algorithm='viterbi')
gmm_hmm_viterbi.fit(functional_signals, lengths)
l = gmm_hmm_viterbi.predict(functional_signals, lengths)
GM12878_dataset.write_annotation(l, 'HMM_viterbi_functional', annotation_dir)

gmm_hmm_viterbi = hmm.GaussianHMM(n_components=k, covariance_type = 'full', algorithm='viterbi')
gmm_hmm_viterbi.fit(structural_signals, lengths)
l = gmm_hmm_viterbi.predict(structural_signals, lengths)
GM12878_dataset.write_annotation(l, 'HMM_viterbi_structural', annotation_dir)

gmm_hmm_viterbi = hmm.GaussianHMM(n_components=k, covariance_type = 'full', algorithm='viterbi')
gmm_hmm_viterbi.fit(signals, lengths)
l = gmm_hmm_viterbi.predict(signals, lengths)
GM12878_dataset.write_annotation(l, 'HMM_viterbi_combined', annotation_dir)

# HMM (MAP)

In [24]:
gmm_hmm_map = hmm.GaussianHMM(n_components=k, covariance_type = 'full', algorithm='map')
gmm_hmm_map.fit(functional_signals, lengths)
l = gmm_hmm_map.predict(functional_signals, lengths)
GM12878_dataset.write_annotation(l, 'HMM_map_functional', annotation_dir)

gmm_hmm_map = hmm.GaussianHMM(n_components=k, covariance_type = 'full', algorithm='map')
gmm_hmm_map.fit(structural_signals, lengths)
l = gmm_hmm_map.predict(structural_signals, lengths)
GM12878_dataset.write_annotation(l, 'HMM_map_structural', annotation_dir)

gmm_hmm_map = hmm.GaussianHMM(n_components=k, covariance_type = 'full', algorithm='map')
gmm_hmm_map.fit(signals, lengths)
l = gmm_hmm_map.predict(signals, lengths)
GM12878_dataset.write_annotation(l, 'HMM_map_combined', annotation_dir)

# GMM-GBR (short)

In [25]:
short_gmm_mp_args = {"i": functional_signals_path, "e": short_hic_path, "g": bins_path, "r": 1, "n": k, 
              "il": 5, "ml": 5, "w": 1, "o": 'GM12878_res100000_mp', "mp": '../../alternatives/segway-gbr-master/measureProp/MP_large_scale'}
short_gmm_mp = GMM_MP(short_gmm_mp_args)
short_gmm_mp.write_mp_graph()
short_gmm_mp.GMM_train()
short_gmm_mp.GMM_inference(0)
for l in range(10):
    short_gmm_mp.write_mp_post(l+1)
    short_gmm_mp.run_measure_prop(l+1)
    short_gmm_mp.read_mp_post(l+1)
    short_gmm_mp.GMM_inference(l+1)
    
for i in range(10):
    GM12878_dataset.write_annotation(short_gmm_mp.iters_labels[i], 'short_gbr_{}'.format(i), 
                                    os.path.join(annotation_dir, 'short_gbr'))

writing mp graph file...
writing mp transduction file...


# GMM-GBR (long)

In [29]:
long_gmm_mp_args = {"i": functional_signals_path, "e": long_hic_path, "g": bins_path, "r": 1, "n": k, 
              "il": 5, "ml": 5, "w": 1, "o": 'GM12878_res100000_mp', "mp": '../../alternatives/segway-gbr-master/measureProp/MP_large_scale'}
long_gmm_mp = GMM_MP(long_gmm_mp_args)
long_gmm_mp.write_mp_graph()
long_gmm_mp.GMM_train()
long_gmm_mp.GMM_inference(0)
for l in range(10):
    long_gmm_mp.write_mp_post(l+1)
    long_gmm_mp.run_measure_prop(l+1)
    long_gmm_mp.read_mp_post(l+1)
    long_gmm_mp.GMM_inference(l+1)
    
for i in range(10):
    GM12878_dataset.write_annotation(long_gmm_mp.iters_labels[i], 'long_gbr_{}'.format(i), 
                                    os.path.join(annotation_dir, 'long_gbr'))

writing mp graph file...
writing mp transduction file...


# GMM-MRF (short)

In [31]:
mrf_signals = mrf_util.readData(functional_signals_path)
mrf_bins = mrf_util.readBedGraph(bins_path)
n = mrf_signals.shape[0]

In [32]:
mrf_short_hic = mrf_util.create_hic_matrix(short_hic_path, n, True)
short_hmrf = MarkovRandomField(n, mrf_short_hic, mrf_signals, 'mrf', k, 10, 0.01)
short_hmrf.init_gmm()
short_hmrf.init_trans()
short_hmrf.solve()

In [33]:
for i in range(10):
    GM12878_dataset.write_annotation(short_hmrf.iters_labels[i], 'short_hmrf_{}'.format(i), 
                                    os.path.join(annotation_dir, 'short_hmrf'))

# GMM-MRF (long)

In [34]:
mrf_long_hic = mrf_util.create_hic_matrix(long_hic_path, n, True)
long_hmrf = MarkovRandomField(n, mrf_long_hic, mrf_signals, 'mrf', k, 10, 0.01)
long_hmrf.init_gmm()
long_hmrf.init_trans()
long_hmrf.solve()

In [35]:
for i in range(10):
    GM12878_dataset.write_annotation(long_hmrf.iters_labels[i], 'long_hmrf_{}'.format(i), 
                                    os.path.join(annotation_dir, 'long_hmrf'))

# Segway

In [8]:
from scipy.stats import entropy
hic = np.loadtxt(hic_path, dtype = int)
def labels_stat(hic, labels):
    first_label = [labels[p] for p in hic[:,0]]
    second_label = [labels[p] for p in hic[:,1]]
    same_ratio = sum([f==s for f,s in zip(first_label,second_label)])/len(first_label)
    _, label_counts = np.unique(labels, return_counts = True)
    labels_entropy = entropy(label_counts, base=None)
    return {'same_ratio': same_ratio, 'labels_entropy': labels_entropy}
for l in range(10):
    print('label stats at iteration {}: {}'.format(l, labels_stat(hic, hmrf.iters_labels[l])))

In [10]:
for l in range(10):
    print('label stats at iteration {}: {}'.format(l, labels_stat(hic, hmrf.iters_labels[l])))

label stats at iteration 0: {'same_ratio': 0.4619559936142153, 'labels_entropy': 2.0545440390338894}
label stats at iteration 1: {'same_ratio': 0.6298604844867078, 'labels_entropy': 1.6467841980619295}
label stats at iteration 2: {'same_ratio': 0.6926910529603665, 'labels_entropy': 1.3694052968864807}
label stats at iteration 3: {'same_ratio': 0.7316026931352815, 'labels_entropy': 1.1868553103027182}
label stats at iteration 4: {'same_ratio': 0.7750537932949261, 'labels_entropy': 1.0448920160823907}
label stats at iteration 5: {'same_ratio': 0.8198098146734226, 'labels_entropy': 0.956859175876129}
label stats at iteration 6: {'same_ratio': 0.8715207885055876, 'labels_entropy': 0.8098756344707698}
label stats at iteration 7: {'same_ratio': 0.9196501700562226, 'labels_entropy': 0.6422419811272213}
label stats at iteration 8: {'same_ratio': 0.9524120219337822, 'labels_entropy': 0.5105077865711872}
label stats at iteration 9: {'same_ratio': 0.9784410356076907, 'labels_entropy': 0.414693084